<a href="https://colab.research.google.com/github/jobellet/detect_CS/blob/master/train_cs_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**STEP 2: TRAIN A NETWORK**

## First of all, drag all the files you want to use for training in the folder : uneye/work/TrainYourNetwork/

Once all the files are uploaded, you can run the next cell

## The cell bellow authorizes google to modify files on your drive
- run the cell
- clic on the link
- select your account
- copy the code
- paste it in the empty field
- press enter

In [0]:
# Make your google drive accessible to colab
from google.colab import drive
drive.mount('/gdrive')

## The next cell installs all necessary dependencies



In [0]:
Path_to_UnEye_folder = '/gdrive/My Drive/uneye/' 

# to do math operations
import numpy as np
import scipy.io as io

# go to the uneye folder
import os
os.chdir(Path_to_UnEye_folder)

# to read .mat files
!pip install mat4py
import mat4py
# to do read .pkl files
import pandas as pd

# to do clustering
!pip install hdbscan

import uneye
from detect_CS import *

# The next cell trains the network
## Change the variables of the next cell as described in the comments
## Then run the cell

In [0]:
# Set all the following parameters before running the cell
field_LFP = ['lfp']         # in .mat files where the LFP data is in the variable lfp then set: field_LFP = ['lfp']
                            # in .mat files where the LFP data is in the structure Data.lfp then set: field_LFP = ['Data','lfp']
                            # you can use either the LFP signal or the wide-band signal
field_high_pass = ['High_Pass'] # same comment as above
field_label = ['Labeled_Data'] # same comment as above

# set the name of the weights of the network
weights_name = 'my_weight_name' # the weights are going to be saved in the folder /training

# set sample frequency of your recording
sampfreq = 25000 # in Hz

# size of the convolutional and maxpooling operations
ks = 9 # convolution kernel size 
mp = 7 # maxpooling size
# the number of bins (nb) taken into account by the network is given by
# nb = mp**2+mp**2*ks+(mp*ks)-mp+2*ks-2 


########################################################################################################################################################################################
training_folder = '/gdrive/My Drive/uneye/work/TrainYourNetwork/'
training_files = os.listdir(training_folder)

bigLFP = []
bigHIGH = []
bigLabels = []
for tf in training_files:
    LFP,High,Labels,_ = load_data(filename = training_folder+tf,field_LFP = field_LFP,field_high_pass = field_high_pass, field_label = field_label)
    interval_inspected = create_random_intervals(sampfreq,LFP,Labels)
    compLFP,compHIGH,compLabels = concatenate_segments(LFP,High,interval_inspected,Labels)
    bigLFP = np.concatenate((bigLFP,compLFP))
    bigHIGH = np.concatenate((bigHIGH,compHIGH))
    bigLabels = np.concatenate((bigLabels,compLabels))
val_samples = np.max(np.round(.1*len(bigLFP)/sampfreq),1)
model = uneye.DNN(max_iter = 3000, ks=ks,mp=mp, weights_name=weights_name,
                  sampfreq = sampfreq,augmentation = False,val_samples = val_samples, doDiff = False)
model.train(bigLFP,bigHIGH,bigLabels)